In [1]:
#import 
import pandas as pd
import numpy as np
import re, nltk        
from nltk.stem.porter import PorterStemmer
from sklearn.svm import LinearSVC
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import *


In [47]:
#read data
df1 = pd.read_csv('./data.csv')
df2 = pd.read_csv('./train.csv')
# Predict actual test data
df3 = pd.read_csv('./Test_Data.csv')

In [48]:
print(df1.head(2))
print(df2.head(2))
print(df3.head(2))

  Document_Id                                               Text
0  Document_0    Personal view of author on goals and content...
1  Document_1    The phenomenon of quantum number fractionali...
       document_id  category
0   Document_51509        23
1  Document_120711         3
       document_id  category
0  Document_122405       NaN
1   Document_82322       NaN


In [55]:
#Join Data
df = df2.join(df1.set_index('Document_Id'), on='document_id')
df = df.drop(df.columns[0], axis=1)

#Join Data
pred_df = df3.join(df1.set_index('Document_Id'), on='document_id')
test_id = pred_df.ix[:,0]
test_data_df = pred_df.drop(pred_df.columns[1], axis=1)

In [56]:
#Splitting Training and testing Data 
# Since Data size is too Large , using approximately 20% of the data.
train_data_df = df.ix[:20000,:]
print(train_data_df.head())
print(test_id.head())
test_data_df = test_data_df.drop(test_data_df.columns[0], axis=1)
print(test_data_df.head())

   category                                               Text
0        23    We study an impartial game introduced by And...
1         3    We have used a first-principles-based scheme...
2        23    Topological indices play an important role i...
3        34    We introduce a category of cluster algebras ...
4        30    We present a general result of simultaneous ...
0    Document_122405
1     Document_82322
2     Document_53999
3    Document_144615
4     Document_90227
Name: document_id, dtype: object
                                                Text
0    We perform real-time lattice simulations of ...
1    We study a discrete-time duplication-deletio...
2    We give a solution to the problem of filling...
3    Matrix variate beta (MVB) distributions are ...
4    We study curvature flows in the locally homo...


In [57]:
#Using Porter Stemmer
stemmer = PorterStemmer()

#function to stem text tokens
def stem_tokens(tokens, stemmer):
    stemmed = []
    for item in tokens:
        stemmed.append(stemmer.stem(item))
    return stemmed

#function to tokenize text elements and removing numbers and punctuation
def tokenize(text):
    
    text = re.sub("[^a-zA-Z]", " ", text)
    tokens = text.split(" ")
    stems = stem_tokens(tokens, stemmer)
    return stems

# Tokenizing and vectorizing the text elements, eleminating stop words, using maximum of 2000 text tokens per document.
vectorizer = TfidfVectorizer(analyzer='word',tokenizer=tokenize,lowercase=True,stop_words ='english',max_features =2000)
corpus_data_features = vectorizer.fit_transform(train_data_df.Text.tolist() + test_data_df.Text.tolist())

#Convering the document term matrix to numpy nd array
corpus_data_features_nd = (corpus_data_features.toarray())
print (corpus_data_features_nd.shape)




(49734, 2000)


In [58]:
# Model Declaration
# L2 regularization with hinged as the loss function and amount of regularization 0.7
my_model = LinearSVC(penalty = 'l2',dual = True,C=0.7,loss='hinge') 

# Fit model with train and test data
my_model = my_model.fit(X=corpus_data_features_nd[0:len(train_data_df)], y=train_data_df.category)
test_pred = my_model.predict(corpus_data_features_nd[len(train_data_df):])


In [78]:
test_id = pred_df.ix[:,0]
test_id = test_id.to_frame()
test_pred = pd.Series(test_pred)

In [81]:
test_id['category'] = test_pred
test_id

,document_id,category
0,Document_122405,18
1,Document_82322,32
2,Document_53999,25
3,Document_144615,36
4,Document_90227,25
5,Document_56018,25
6,Document_127410,11
7,Document_54331,28
8,Document_93775,31
9,Document_5601,17


In [82]:
test_pred.to_csv("./predicted_category.csv")